#### Business Analytics FHDW 2025
## Aufgabe

Eine Personalberatung untersucht die Einflüsse der Erfahrung und Ausbildung von Systemadmins auf deren Fähigkeit, bestimmte Aufgaben in begrenzter Zeit zu lösen. Die Daten von 75 Admins finden sich in `SystemAdministrators.csv`: Die Variable *Experience* zeigt die Vollzeitbeschäftigung als Admin in Monaten, *Training* erworbene, fachlich relevante Credit Points, die Zielvariable *Completed task* zeigt, ob die Aufgaben in der verfügbaren Zeit gelöst wurden.

1. Generieren Sie einen Scatterplot, der (z. B. farblich) illustriert, wer die Aufgaben gelöst hat und wer nicht. Welcher Prädiktor ist potentiell verwendbar für eine Vorhersage der erfolgreichen Aufgabenlösung?

2. Implementieren Sie ein logistisches Regressionsmodell mit beiden Prädiktoren, das den gesamten Datensatz für das Training nutzt. Was ist der Prozentsatz der erfolgreichen Admins, die fälschlich als gescheitert eingeordnet werden, wenn Sie die Trainingsdaten auch für die Vorhersage verwenden?

3. Sollten Sie den *cutoff* höher oder niedriger setzen, um den Prozentsatz aus 2 zu senken?

4. Wie viele Monate Erfahrung muss ein Admin mit 4 Credits besitzen, damit die Erfolgswahrscheinlichkeit für die zeitige Aufgabenlösung 0.5 erreicht?

### Zu 1:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pylab as plt
from dmba import classificationSummary

admins_df = pd.read_csv('./Daten/SystemAdministrators.csv')
admins_df

In [ ]:
colors = np.where(admins_df["Completed task"]=='Yes','g','r')
print(colors)
splot = admins_df.plot.scatter(x='Experience',y='Training', c=colors)

Mit steigender Ausbildung steigt der Anteil der erfolgreichen Admins, mit steigender Erfahrung ebenso. Auf einen ersten Blick könnten also beide Variablen als Prädiktoren für den Erfolg dienen.

### Zu 2:

Zunächst wandeln wir die kategorische Variable in einen numerischen Dummy um.

In [ ]:
admins_df = pd.get_dummies(admins_df, drop_first=True)
admins_df

In [ ]:
y = admins_df['Completed task_Yes']
X = admins_df[['Experience', 'Training']] # oder drop(columns=...
log_reg = LogisticRegression()
log_reg.fit(X, y)
              
log_reg_predictions = log_reg.predict(X)

classificationSummary(y, log_reg_predictions)

Insgesamt haben 15 Admins die Aufgaben zeitgerecht bewältigt. Ein Drittel davon (5/15) sind aber fälschlich als erfolglos klassifiziert worden.

### Zu 3:

Wir implementieren die Vorhersage mit expliziten Schwellwerten:

In [ ]:
predictions = np.where(log_reg.predict_proba(X) >= 0.5, 1, 0)[:,1]
classificationSummary(y, predictions)

In [ ]:
predictions = np.where(log_reg.predict_proba(X) >= 0.3, 1, 0)[:,1]
classificationSummary(y, predictions)

Senken wir den *cutoff*-Wert, werden bei gleich bleibender Gesamtgenauigkeit weniger erfolgreiche Admins falsch eingeordnet.

### Zu 4:

Es gibt verschiedene Möglichkeiten, die notwendige Erfahrung zu ermitteln.

Um einen Untersuchungsbereich zu definieren, ermitteln wir die obere Grenze der Erfahrung:

In [ ]:
print(max(admins_df.Experience))

Dann können wir eine erste Näherung für den gesuchten Wert bestimmen:

In [ ]:
experience = np.arange(0, 14, .01) # Aufgerundete obere Grenze von eben.
test_df = pd.DataFrame({'Experience': experience, 'Training': 4})
print(test_df)
# Nun lassen wir uns den ersten Wert ausgeben, der die geforderte Wahrscheinlichkeit erreicht:
required_experience = experience[np.where(log_reg.predict_proba(test_df)[:,1]>=0.5)[0][0]]
print(required_experience)

Alternativ können wir auch mit den Formeln arbeiten. Die können wir uns aus den Parametern konstruieren.

In [ ]:
coefficients = pd.DataFrame({'coeff': log_reg.coef_[0]}, index=X.columns)
print(log_reg.coef_)
print(coefficients.transpose())

In [ ]:
intercept = log_reg.intercept_[0]
coeff_Experience = log_reg.coef_[0][0]
coeff_Training = log_reg.coef_[0][1]

Z = lambda x : intercept+coeff_Experience*x+coeff_Training*4
P = lambda x : 1/(1+np.exp(-1*Z(x)))
Logit = lambda x : Z(x)

Wir wollen wissen, ab welchem Punkt die Wahrscheinlichkeitsfunktion den Wert 0.5 besitzt. Dazu nutzen wir den Logit:

In [ ]:
np.log(0.5/(1-0.5))

Also müssen wir ermitteln, an welchem Punkt $Z = 0$ ist. Dazu nehmen wir eine Termumformung von $Z$ vor, wie wir das aus den Mathematikveranstaltungen, oder noch aus der Schule kennen:

In [ ]:
Z_null = ((intercept*-1)-coeff_Training*4)/coeff_Experience
print(Z_null)

In [ ]:
P(Z_null)

Grafische Auswertung:

In [ ]:
plt.scatter(experience, log_reg.predict_proba(test_df)[:,1], s=1)
plt.show()